In [2]:
from pydantic import BaseModel
from langchain_core.messages import BaseMessage
from typing import Annotated,Sequence,Optional
import operator

class AgentState(BaseModel):
    messages:Annotated[Sequence[BaseMessage],operator.add]
    search_results: Optional[str] = None
    python_output: Optional[str] = None


from langchain_tavily import TavilySearch
from langchain.tools import tool
import os
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchRun
load_dotenv()
TAVILY_API_KEY=os.getenv("TAVILY_APIKEY")
@tool
def web_search(state: AgentState) -> AgentState:
    """Search the web using Tavily with the user's input."""
    print("Web Search tool called-----")
    query = ""
    for msg in reversed(state.messages):
        if msg.type == "user":
            query = msg.content
            break
    try:
        results = TavilySearchResults(max_results=4,tavily_api_key=TAVILY_API_KEY).invoke(query)
        print(results)
        return state.model_copy(update={"search_results": results})
    except Exception as e:
        print(e)

from langgraph.prebuilt import create_react_agent
from azure.identity import DefaultAzureCredential,get_bearer_token_provider
from langchain_openai import AzureChatOpenAI

token_provider = get_bearer_token_provider(DefaultAzureCredential(),"https://cognitiveservices.azure.com/.default")
chat_model= AzureChatOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider= token_provider,
    model= "gpt-4o-mini"
)



def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )


research_agent = create_react_agent(
    chat_model,
    tools=[web_search],
    prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague."
    ),
)

In [3]:
research_agent.invoke({"messages":"What is the latest news"})

Web Search tool called-----
[{'title': 'ABC News - Breaking News, Latest News and Videos', 'url': 'https://abcnews.go.com/', 'content': "### Thailand seizes over 2 tons of meth worth more than $90.8 million\n\n### New IOC president Kirsty Coventry inaugurated to start 8-year leadership\n\n### Mideast governments condemn Syria church bombing as death toll jumps to 25\n\n### Madeleine McCann suspect could be freed from prison early after donor pays fine\n\n### London's secret wartime tunnels set to draw tourists with spy museum and bar\n\n### Good Morning America [...] ### Newlywed couple's honeymoon in US Virgin Islands ends with her ICE detention\n\n### Florida condo owners will get financial relief under a new law\n\n### Veteran race car driver wants to make history at Pikes Peak\n\n### Summer travel forecast: The cheapest days and the busiest days to fly\n\n### Fat Joe accused of sex trafficking, fraud in new lawsuit\n\n### Dad arrested for son's death after allegedly leaving him in 

{'messages': [HumanMessage(content='What is the latest news', additional_kwargs={}, response_metadata={}, id='7a8197e0-636d-48a3-ac94-f6cdc0792f33'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yQZ7nAzikD9RVyF4NvmOhh6q', 'function': {'arguments': '{"state":{"messages":[{"content":"latest news","type":"user"}]}}', 'name': 'web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 239, 'total_tokens': 266, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_57db37749c', 'id': 'chatcmpl-BlhoTaYqOPiY5Ya4GXGagIhBcYhkp', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'